In [ ]:
# Settings
from PIL import Image, ImageEnhance
import os
import numpy as np
from tqdm import tqdm
import pathlib
import shutil
import cv2

# Set the directory containing the images you want to modify
directory = "../Datasets/Replica/"
output_dir = "../Datasets/Replica_tmp/"
skip_interval = 40
interval = 10

# Set the size of the facula
size = 100

In [ ]:
#RANDOM LOCAL ILLUMINATION

# find all maps
maps = [x for x in os.listdir(directory) if not x.startswith(".") and os.path.isdir(os.path.join(directory, x))]

for map in maps:
    print(f"Processing map {map}...")

    map_path = os.path.join(directory, map, "results")
    output_map_path = os.path.join(output_dir, map, "results")
    # copy traj.txt
    pathlib.Path(output_map_path).mkdir(parents=True, exist_ok=True)
    shutil.copy(os.path.join(map_path, "../traj.txt"), os.path.join(output_map_path, "../traj.txt"))

    # Loop through all the files in the directory
    img_files = [x for x in os.listdir(map_path)]
    img_files.sort()
    x = None
    for img_file in tqdm(img_files):
        image_path = os.path.join(map_path, img_file)
        image_save_path = os.path.join(output_map_path, img_file)
        id = int(img_file.split(".")[0][5:])
        # just copy depth maps
        if img_file.startswith("depth") or id % (skip_interval + interval) < skip_interval:
            shutil.copy(image_path, image_save_path)            
            continue
            
        # Open the image
        img = cv2.imread(image_path)

        height, width = img.shape[:2]

        # Generate a random position for the facula
        if (id % (skip_interval + interval)) == skip_interval:
            x = np.random.randint(size, width - size)
            y = np.random.randint(size, height - size)

            # Generate a random color for the facula
            color = np.random.randint(0, 256, size=(1)).repeat(3).reshape(1, 1, 3)

        # Create a circular mask for the facula
        mask = np.zeros((size*2, size*2, 3), dtype=np.uint8)
        cv2.circle(mask, (size, size), size, (1,1,1), -1)
        mask = mask[:size*2, :size*2]
        # Add the facula to the image
        img[y-size:y+size, x-size:x+size] = color * mask + img[y-size:y+size, x-size:x+size] * (1 - mask)

        # Save the modified image with a new filename
        cv2.imwrite(image_save_path, img)

print("Done")